- `gpu_memory_utilization`
    - Weights
    - activation
    - kv cache
- advanced parameters
    - dynamic split fuse 默认 able（除非 disable）
    - 开启之后，可以显著提升 TTFT（time to first token）

## quant

- vllm vs. ollama
    - vllm: awq, gptq
    - ollama: gguf

## benchmark_serving.py

- benchmark metircs
    - ttft: time to first token